<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/delta_hedge_portfolios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.stats import norm

# -------------
# Example:
#   Long a call strike at 50,  a put strike at 55, and a unit of underlying.
# -------------

# example parameters
S = 50
K_call = 50
K_put = 55
T = 1
sigma = 0.4
r = 0.02
q = 0.03

# call

K = K_call

d1 = (np.log(S/K) + (r-q+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)
qT = np.exp(-q*T)
rT = np.exp(-r*T)

N1 = norm.cdf(d1)
N2 = norm.cdf(d2)
call_premium = qT * S * N1 - rT * S * N2
call_delta = N1
call_intercept =  - ((call_delta * S) - call_premium)

# put

K = K_put

d1 = (np.log(S/K) + (r-q+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)
qT = np.exp(-q*T)
rT = np.exp(-r*T)

N1 = norm.cdf(-d1)
N2 = norm.cdf(-d2)
put_premium = rT * K * N2 - qT * S * N1
put_delta = norm.cdf(d1) - 1
put_intercept =  - ((put_delta * S) - put_premium)


# sum up each asset in the portfolio

port_delta = call_delta + put_delta + 1  # +1 because we also hold a long position of a unit of underlying
port_intercept = -((port_delta * S) - (call_premium + put_premium + S))

print('The delta for the portfolio is', port_delta.round(2), 'when the underlying price is', S, end='.\n')
print('The delta-hedge portfolio shorts', abs(port_delta.round(2)),
      'shares of the underlying and saves', port_intercept.round(2), 'cash.')

The delta for the portfolio is 1.04 when the underlying price is 50.
The delta-hedge portfolio shorts 1.04 shares of the underlying and saves 16.37 cash.
